In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.utils import to_categorical

In [14]:
text = open('../dataset/text.txt', "r", encoding="utf-8").read().lower()

chars = sorted(list(set(text)))
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for i, c in enumerate(chars)}

SEQ_LENGTH = 100
VOCAB_SIZE = len(chars)

data = [char_to_idx[c] for c in text]

In [15]:
X = []
y = []

for i in range(len(data) - SEQ_LENGTH):
    X.append(data[i : i + SEQ_LENGTH])
    y.append(data[i + SEQ_LENGTH])

X = np.array(X).reshape(len(X), SEQ_LENGTH)
y = np.array(y)

y = to_categorical(y, num_classes=VOCAB_SIZE)

In [16]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, 64, input_length=SEQ_LENGTH))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(VOCAB_SIZE, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam")

In [17]:
class TextGeneratorCallback(tf.keras.callbacks.Callback):
    def __init__(self, seed, char_to_idx, idx_to_char, seq_length=100, temps=[0.2, 0.5, 1.0]):
        super().__init__()
        self.seed = seed.lower()
        self.char_to_idx = char_to_idx
        self.idx_to_char = idx_to_char
        self.seq_length = seq_length
        self.temps = temps

    def on_epoch_end(self, epoch, logs=None):
        print(f"\n--- Epoch {epoch+1} ---")
        for temp in self.temps:
            generated = self.seed
            start_idx = max(0, len(generated) - self.seq_length)
            generated = generated[start_idx:]
            
            for _ in range(200):
                x = np.array([self.char_to_idx[c] for c in generated[-self.seq_length:]])
                x = x.reshape(1, -1)
                preds = self.model.predict(x, verbose=0)[0]

                preds = np.log(preds) / temp
                exp_preds = np.exp(preds)
                preds = exp_preds / np.sum(exp_preds)
                next_idx = np.random.choice(len(self.char_to_idx), p=preds)
                generated += self.idx_to_char[next_idx]
            
            print(f"Temperature={temp}:\n{generated}\n")

In [18]:
seed_text = 'In the midway of this our mortal life'
temps = [0.2, 0.5, 1.0]

text_gen_callback = TextGeneratorCallback(
    seed_text, 
    char_to_idx, 
    idx_to_char, 
    seq_length=SEQ_LENGTH, 
    temps=temps,
)

model.fit(
    X, 
    y, 
    epochs=10, 
    batch_size=128, 
    validation_split=0.2, 
    callbacks=[text_gen_callback],
)

Epoch 1/10


435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step - loss: 2.9646
--- Epoch 1 ---
Temperature=0.2:
in the midway of this our mortal life the the the the the the the the the the the the the the the the the the the the fere the the he the the the the the the the the the the the the the the the the the the the the the the the the the the

Temperature=0.5:
in the midway of this our mortal life theun in in the the the meed,
thor sre the fom sann a ans sith the ce te sod laghe inet spas the ing mes sos the orert the heen ins he neine feeregh the the mo the the an, ant se sath the the se hire

Temperature=1.0:
in the midway of this our mortal lifeud racrh beonde fase,

ase,
thure te than’ nufed boto.
thopet,  tirl,”. suuss ipret bundt, there nerletidtumcibt insny
lrerl giry “sietthr les af the fnerld
ghad pinth hute
ioth the lamd, k the ipo pe

435/435 ━━━━━━━━━━━━━━━━━━━━ 217s 493ms/step - loss: 2.9641 - val_loss: 2.3759
Epoch 2/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step - loss: 2.3356
--- Epoch 

In [ ]:
import json

# Загрузка данных из исходного JSON
with open('E:\User\Downloads\list_of_moscow_metro_stations.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Создание базовой структуры GeoJSON
geojson = {
    "type": "FeatureCollection",
    "features": []
}

# Обработка каждой станции
for station in data:
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [float(station["Longitude"]), float(station["Latitude"])]
        },
        "properties": {
            "name": station["Name"],
            "line": station["Line"],
            "line_color": f"#{station['LineColor']}",  # Добавляем # к цвету
            "order": station["Order"]
        }
    }
    geojson["features"].append(feature)

# Сохранение результата
with open('moscow_metro.geojson', 'w', encoding='utf-8') as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)